In [63]:
import pandas as pd
import requests
import timeit
from io import StringIO
from datetime import timedelta, date, datetime
from IPython.display import clear_output

In [64]:
# get two dates as strings, yield dates in range as strings
def daterange(start_date, end_date):
    for n in range(int((end_date - start_date).days)):
        yield (start_date + timedelta(n)).strftime('%Y-%m-%d')

days = list(daterange(datetime(2015, 1, 1), datetime(2020, 1, 1)))

In [65]:
result_df = pd.DataFrame(columns=['RecordType', 'SettlementDate', 'SettlementPeriod', 'Zone', 'TimePublished', 'Quantity'])

In [66]:
consumption_forecast_url = 'https://api.bmreports.com/BMRS/FORDAYDEM/v1?APIKey=ozdvc52r48xnxum&ZoneIdentifier=N&FromDate={}&ToDate={}&ServiceType=csv'

for i in range(len(days)):
    day_from = day_to = days[i]

    r = requests.get(consumption_forecast_url.format(day_from, day_to))
    response_text = r.text.split('\n')[1:-1]
    response_text = '\n'.join(response_text)
    
    response_df = pd.read_csv(StringIO(response_text), header=None)
    response_df.columns = ['RecordType', 'SettlementDate', 'SettlementPeriod', 'Zone', 'TimePublished', 'Quantity']
    
    result_df = result_df.append(response_df.loc[response_df.RecordType == 'DANF'])
    
    clear_output(wait=True)
    print(f'{round(i/len(days), 2)*100}% processed, shape: {result_df.shape}', flush=True)

1.0% processed, shape: (87648, 6)


In [67]:
result_df.to_csv('uk_demand_forecast.csv')

In [68]:
result_df

,RecordType,SettlementDate,SettlementPeriod,Zone,TimePublished,Quantity
0,DANF,20150101,1,N,20141231234700,29400
1,DANF,20150101,2,N,20150101001700,29800
2,DANF,20150101,3,N,20150101004700,28920
3,DANF,20150101,4,N,20150101011700,27932
4,DANF,20150101,5,N,20150101014700,26868
...,...,...,...,...,...,...
43,DANF,20191231,44,N,20191231211500,29773
44,DANF,20191231,45,N,20191231214500,28488
45,DANF,20191231,46,N,20191231221500,27817
46,DANF,20191231,47,N,20191231224500,26872
